<a href="https://colab.research.google.com/github/Ahmad08017928/Ahmad08017928/blob/main/Submision_akhir_dicoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import Library**

In [ ]:
# Library yang sering digunakan
import os, shutil, json
import zipfile
import random
from random import sample
import shutil
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm as tq

In [ ]:
# Libraries untuk pemrosesan data gambar
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import skimage
from skimage import io
from skimage.transform import resize
from skimage.transform import rotate, AffineTransform, warp
from skimage import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

In [ ]:
# Libraries untuk pembangunan model
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import InputLayer, Conv2D, SeparableConv2D, MaxPooling2D, MaxPool2D, Dense, Flatten, Dropout, Add, Activation, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout, Input, Add, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#**Mengkoneksi-kan ke Kaggle**

In [ ]:
# Menyimpan API token di ~/.kaggle/kaggle.json
os.makedirs('/root/.kaggle/', exist_ok=True)

api_token = {"username": "what6476", "key": "6eda561fb1794c48b6140e3ade630090"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

# Atur izin
os.chmod('/root/.kaggle/kaggle.json', 600)

#**Install Kaggle dan Unzip file yang di tuju**

In [ ]:
!pip install -q kaggle

In [ ]:
# Download dan unzip dataset
!kaggle datasets download -d tolgadincer/labeled-chest-xray-images
!unzip labeled-chest-xray-images.zip

Dataset URL: https://www.kaggle.com/datasets/tolgadincer/labeled-chest-xray-images
License(s): other
labeled-chest-xray-images.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  labeled-chest-xray-images.zip
replace chest_xray/test/NORMAL/NORMAL-1049278-0001.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Direktori awal untuk train dan test
train_dir = "chest_xray/train"
test_dir = "chest_xray/test"

# Direktori baru untuk dataset gabungan
combined_dir = "chest_xray/dataset"

In [ ]:
# Buat direktori baru untuk dataset gabungan
os.makedirs(combined_dir, exist_ok=True)

In [ ]:
# Salin file dan folder dari train
for category in os.listdir(train_dir):
    category_dir = os.path.join(train_dir, category)
    if os.path.isdir(category_dir):
        shutil.copytree(category_dir, os.path.join(combined_dir, category), dirs_exist_ok=True)

# Salin file dan folder dari test
for category in os.listdir(test_dir):
    category_dir = os.path.join(test_dir, category)
    if os.path.isdir(category_dir):
        shutil.copytree(category_dir, os.path.join(combined_dir, category), dirs_exist_ok=True)

#**Data Checking**

In [ ]:
# Membuat kamus yang menyimpan gambar untuk setiap kelas dalam data
lung_image = {}

# Tentukan path sumber train
path = "chest_xray/"
path_sub = os.path.join(path, "dataset")
for i in os.listdir(path_sub):
    lung_image[i] = os.listdir(os.path.join(path_sub, i))

In [ ]:
# Menampilkan secara acak 5 gambar di bawah setiap dari 2 kelas dari data.
# Anda akan melihat gambar yang berbeda setiap kali kode ini dijalankan.
path_sub = "chest_xray/dataset/"

# Menampilkan secara acak 5 gambar di bawah setiap kelas dari data latih
fig, axs = plt.subplots(len(lung_image.keys()), 5, figsize=(15, 15))

for i, class_name in enumerate(os.listdir(path_sub)):
    images = np.random.choice(lung_image[class_name], 5, replace=False)

    for j, image_name in enumerate(images):
        img_path = os.path.join(path_sub, class_name, image_name)
        img = Image.open(img_path).convert("L")  # Konversi menjadi skala keabuan
        axs[i, j].imshow(img, cmap='gray')
        axs[i, j].set(xlabel=class_name, xticks=[], yticks=[])


fig.tight_layout()

In [ ]:
# Definisikan path sumber
lung_path = "chest_xray/dataset/"

# Buat daftar yang menyimpan data untuk setiap nama file, path file, dan label dalam data
file_name = []
labels = []
full_path = []

# Dapatkan nama file gambar, path file, dan label satu per satu dengan looping, dan simpan sebagai dataframe
for path, subdirs, files in os.walk(lung_path):
    for name in files:
        full_path.append(os.path.join(path, name))
        labels.append(path.split('/')[-1])
        file_name.append(name)

distribution_train = pd.DataFrame({"path":full_path, 'file_name':file_name, "labels":labels})

#**Data Augmentation**

In [ ]:
# Membuat fungsi untuk melakukan rotasi berlawanan arah jarum jam
def anticlockwise_rotation(img):
    img = cv2.cvtColor(img, 0)
    img = cv2.resize(img, (224,224))
    sudut = random.randint(0,180)
    return rotate(img, sudut)

# Membuat fungsi untuk melakukan rotasi searah jarum jam
def clockwise_rotation(img):
    img = cv2.cvtColor(img, 0)
    img = cv2.resize(img, (224,224))
    sudut = random.randint(0,180)
    return rotate(img, -sudut)

# Membuat fungsi untuk membalik gambar secara vertikal dari atas ke bawah
def flip_up_down(img):
    img = cv2.cvtColor(img, 0)
    img = cv2.resize(img, (224,224))
    return np.flipud(img)

# Membuat fungsi untuk memberikan efek peningkatan kecerahan pada gambar
def add_brightness(img):
    img = cv2.cvtColor(img, 0)
    img = cv2.resize(img, (224,224))
    img = adjust_gamma(img, gamma=0.5,gain=1)
    return img

# Membuat fungsi untuk memberikan efek blur pada gambar
def blur_image(img):
    img = cv2.cvtColor(img, 0)
    img = cv2.resize(img, (224,224))
    return cv2.GaussianBlur(img, (9,9),0)

# Membuat fungsi untuk memberikan efek pergeseran acak pada gambar
def sheared(img):
    img = cv2.cvtColor(img, 0)
    img = cv2.resize(img, (224,224))
    transform = AffineTransform(shear=0.2)
    shear_image = warp(img, transform, mode="wrap")
    return shear_image

# Membuat fungsi untuk melakukan pergeseran melengkung pada gambar
def warp_shift(img):
    img = cv2.cvtColor(img, 0)
    img = cv2.resize(img, (224,224))
    transform = AffineTransform(translation=(0,40))
    warp_image = warp(img, transform, mode="wrap")
    return warp_image

In [ ]:
# Membuat variabel transformasi yang akan menyimpan semua proses pra-pemrosesan yang telah dilakukan sebelumnya
transformations = { 'rotate anticlockwise': anticlockwise_rotation,
                    'rotate clockwise': clockwise_rotation,
                    'warp shift': warp_shift,
                    'blurring image': blur_image,
                    'add brightness' : add_brightness,
                    'flip up down': flip_up_down,
                    'shear image': sheared
                  }

images_path="chest_xray/dataset/NORMAL" # Path untuk gambar asli
augmented_path="chest_xray/dataset/NORMAL" # Path untuk gambar yang sudah diaugmentasi
images=[] # Penyimpanan gambar yang telah melalui pra-pemrosesan

# Baca nama gambar dari folder dan tambahkan path ke dalam array "images"
for im in os.listdir(images_path):
    images.append(os.path.join(images_path,im))

# Jumlah gambar yang akan ditambahkan dengan hasil transformasi augmentasi, jumlahnya disesuaikan sesuai kebutuhan
# Variabel untuk melakukan iterasi sampai jumlah gambar yang ditentukan dalam images_to_generate
images_to_generate=2000
i=1

while i<=images_to_generate:
    image=random.choice(images)
    try:
        original_image = io.imread(image)
        transformed_image=None
        n = 0      # Variabel untuk melakukan iterasi sampai jumlah transformasi yang akan diterapkan
        transformation_count = random.randint(1, len(transformations)) # Pilih jumlah transformasi acak yang akan diterapkan pada gambar

        while n <= transformation_count:
            key = random.choice(list(transformations)) # Secara acak memilih dan memanggil metode
            transformed_image = transformations[key](original_image)
            n = n + 1

        new_image_path= "%s/augmented_image_%s.jpg" %(augmented_path, i)
        transformed_image = img_as_ubyte(transformed_image)  # Mengonversi gambar ke format byte yang tidak ditandatangani, dengan nilai dalam [0, 255]
        cv2.imwrite(new_image_path, transformed_image)  # Simpan hasil transformasi augmentasi pada gambar ke path yang ditentukan
        i =i+1
    except ValueError as e:
        print('could not read the',image ,':',e,'hence skipping it.')

In [ ]:
# Definisikan path sumber
lung_path = "chest_xray/dataset/"

# Buat daftar yang menyimpan data untuk setiap nama file, path file, dan label dalam data
file_name = []
labels = []
full_path = []

# Dapatkan nama file gambar, path file, dan label satu per satu dengan looping, dan simpan sebagai dataframe
for path, subdirs, files in os.walk(lung_path):
    for name in files:
        full_path.append(os.path.join(path, name))
        labels.append(path.split('/')[-1])
        file_name.append(name)

distribution_train = pd.DataFrame({"path":full_path, 'file_name':file_name, "labels":labels})

# Plot distribusi gambar di setiap kelas
Label = distribution_train['labels']
plt.figure(figsize = (6,6))
sns.set_style("darkgrid")
plot_data = sns.countplot(Label)

In [ ]:
# Panggil variabel mypath yang menampung folder dataset gambar
mypath= 'chest_xray/dataset/'

file_name = []
labels = []
full_path = []
for path, subdirs, files in os.walk(mypath):
    for name in files:
        full_path.append(os.path.join(path, name))
        labels.append(path.split('/')[-1])
        file_name.append(name)

# Memasukkan variabel yang sudah dikumpulkan pada looping di atas menjadi sebuah dataframe agar rapi
df = pd.DataFrame({"path":full_path,'file_name':file_name,"labels":labels})
# Melihat jumlah data gambar pada masing-masing label
df.groupby(['labels']).size()

In [ ]:
# Variabel yang digunakan pada pemisahan data ini di mana variabel x = data path dan y = data labels

X= df['path']
y= df['labels']

# Split dataset awal menjadi data train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=300)

In [ ]:
# Menyatukan ke dalam masing-masing dataframe
df_tr = pd.DataFrame({'path':X_train,'labels':y_train,'set':'train'})
df_te = pd.DataFrame({'path':X_test,'labels':y_test,'set':'test'})

In [ ]:
# Gabungkan DataFrame df_tr dan df_te
df_all = pd.concat([df_tr, df_te], ignore_index=True)

print('===================================================== \n')
print(df_all.groupby(['set', 'labels']).size(), '\n')
print('===================================================== \n')

# Cek sampel data
print(df_all.sample(5))

# Memanggil dataset asli yang berisi keseluruhan data gambar yang sesuai dengan labelnya
datasource_path = "chest_xray/dataset/"
# Membuat variabel Dataset, tempat menampung data yang telah dilakukan pembagian data training dan testing
dataset_path = "Dataset-Final/"

In [ ]:
for index, row in tq(df_all.iterrows()):
    # Deteksi filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['labels'],row['image'].split('.')[0])

    # Buat direktori tujuan folder
    if os.path.exists(os.path.join(dataset_path,row['set'],row['labels'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['labels']))

    # Tentukan tujuan file
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['labels'],destination_file_name)

    # Salin file dari sumber ke tujuan
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

#**Image Data Generator**

In [ ]:
# Definisikan direktori training dan test
TRAIN_DIR = "Dataset-Final/train/"
TEST_DIR = "Dataset-Final/test/"

train_normal = os.path.join(TRAIN_DIR + '/NORMAL')
train_pneumonia = os.path.join(TRAIN_DIR + '/PNEUMONIA')
test_normal = os.path.join(TEST_DIR + '/NORMAL')
test_pneumonia = os.path.join(TEST_DIR + '/PNEUMONIA')

print("Total number of normal images in training set: ",len(os.listdir(train_normal)))
print("Total number of pneumonic images in training set: ",len(os.listdir(train_pneumonia)))
print("Total number of normal images in test set: ",len(os.listdir(test_normal)))
print("Total number of pneumonic images in test set: ",len(os.listdir(test_pneumonia)))

In [ ]:
# Buat objek ImageDataGenerator yang menormalkan gambar
datagen = ImageDataGenerator(rescale=1/255.,
                             validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(TRAIN_DIR,
                                              batch_size=32,
                                              target_size=(150,150),
                                              color_mode="grayscale",
                                              class_mode='binary',
                                              subset='training',
                                              shuffle=True)

validation_generator = datagen.flow_from_directory(TRAIN_DIR,
                                                   batch_size=32,
                                                   target_size=(150,150),
                                                color_mode="grayscale",
                                                   class_mode='binary',
                                                   subset='validation',
                                                   shuffle=False)

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  batch_size=1,
                                                  target_size=(150,150),
                                                  color_mode="grayscale",
                                                  class_mode='binary',
                                                  shuffle=False)

#**Model ResNet**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout, Add, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import Input, regularizers

# Clear session
tf.keras.backend.clear_session()

####################### Residual Block Function ##################################
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    # First Conv layer
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second Conv layer
    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)

    # Shortcut connection
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

####################### Build ResNet model in Sequential ##################################
model_organ = Sequential()

# Input layer
model_1.add(Input(shape=(150, 150, 1)))

# Initial Conv layer
model_organ.add(Conv2D(32, (3, 3), strides=(1, 1), padding='same', kernel_regularizer=regularizers.l2(0.001)))
model_organ.add(BatchNormalization())
model_organ.add(Activation('relu'))

# First Residual Block
model_organ.add(residual_block(model_1.output, filters=32))

# Second Residual Block with downsampling
model_organ.add(residual_block(model_1.output, filters=64, stride=2))

# Third Residual Block with downsampling
model_organ.add(residual_block(model_1.output, filters=128, stride=2))

# Fourth Residual Block
model_organ.add(residual_block(model_1.output, filters=256))

# Global Average Pooling to reduce dimensions before fully connected layers
model_organ.add(GlobalAveragePooling2D())

# Fully connected layers
model_organ.add(Dense(128, activation='relu'))
model_organ.add(Dropout(0.5))
model_organ.add(Dense(64, activation='relu'))
model_organ.add(Dropout(0.3))

model_organ.add(Dense(1, activation='sigmoid'))

model_organ.compile(optimizer=tf.keras.optimizers.RMSprop(),
                loss='binary_crossentropy',
                metrics=['accuracy'])

# Summary of the Model Architecture
print(model_organ.summary())

In [ ]:
# # tf.keras.backend.clear_session()

# ####################### Init sequential model ##################################
# model_1 = Sequential()

# # ######################### Input layer with Fully Connected Layer ################################
# # 1st Convolutional layer, Batch Normalization layer, and Pooling layer
# model_1.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(150,150,1)))
# model_1.add(BatchNormalization())
# model_1.add(MaxPool2D((2, 2)))

# # 2nd Convolutional layer, Batch Normalization layer, and Pooling layer
# model_1.add(Conv2D(32, (4, 4),padding='same', activation='relu'))
# model_1.add(BatchNormalization())
# model_1.add(MaxPool2D((2, 2)))

# # 3rd Convolutional layer, Batch Normalization layer, and Pooling layer
# model_1.add(Conv2D(32, (7, 7), padding='same', activation='relu'))
# model_1.add(BatchNormalization())
# model_1.add(MaxPool2D((2, 2)))

# # Flatten layer
# model_1.add(Flatten())
# # 1nd Dense Layer
# model_1.add(Dense(128, activation = 'relu'))
# # 1nd Dropout Layer
# model_1.add(Dropout(0.5))
# # 2nd Dense Layer
# model_1.add(Dense(64, activation = 'relu'))
# # 2nd Dropout Layer
# model_1.add(Dropout(0.3))

# # Final Dense layer => For output prediction 1 mean (binary class in dataset), sigmoid for binary cases
# model_1.add(Dense(1, activation='sigmoid'))
# ######################### Fully Connected Layer ################################

# ######################### Compile Model ################################
# model_1.compile(optimizer=tf.keras.optimizers.RMSprop(),
#                 loss='binary_crossentropy',
#                 metrics=['accuracy'])

# # Summary of the Model Architecture
# print(model_1.summary())

In [ ]:
# Menghitung jumlah gambar untuk setiap kelas
total_normal = len(os.listdir(train_normal))
total_pneumonia = len(os.listdir(train_pneumonia))

# Menghitung bobot kelas berdasarkan perbandingan jumlah data
normal_weight = (total_normal + total_pneumonia) / (2 * total_normal)
pneumonia_weight = (total_pneumonia + total_normal) / (2 * total_pneumonia)

class_weights = {0: normal_weight, 1: pneumonia_weight}

# Mendefinisikan callback untuk menghentikan pelatihan berdasarkan akurasi
class StopOnAccuracy(Callback):
    def on_epoch_end(self, epoch, logs=None):
        accuracy_train = logs.get('accuracy')
        accuracy_val = logs.get('val_accuracy')

        if accuracy_train and accuracy_val:
            if accuracy_train > 0.85 and accuracy_val > 0.85:
                print("\nPelatihan dihentikan: Akurasi telah mencapai 85% pada pelatihan dan validasi.")
                self.model.stop_training = True

# Membuat instance callback
stop_training_callback = StopOnAccuracy()


In [ ]:
# count_normal, count_pneumonia = len(os.listdir(train_normal)), len(os.listdir(train_pneumonia))
# weight_0 = (1 / count_normal) * (count_normal + count_pneumonia) / 2.0
# weight_1 = (1 / count_pneumonia) * (count_pneumonia + count_normal) / 2.0

# class_weights = {0 : weight_0, 1 : weight_1}

# %time

# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if(logs.get('accuracy') > 0.85 and logs.get('val_accuracy') > 0.85):
#           print("\nAkurasi telah mencapai >85%")
#           self.model.stop_training = True

# callbacks = myCallback()

# # Fitting / training model
# # history_1 = model_1.fit(train_generator,
# #                         epochs=3,
# #                         batch_size=32,
# #                         validation_data=validation_generator,
# #                         class_weight = class_weights,
# #                         # callbacks=callbacks
# #                         )

# **Grafik akurasi dan loss**

In [ ]:
# Mengambil data akurasi dan loss dari history pelatihan
train_acc = history_1.history['accuracy']
val_acc = history_1.history['val_accuracy']
train_loss = history_1.history['loss']
val_loss = history_1.history['val_loss']

epoch_range = range(len(train_acc))

# Plot Akurasi Pelatihan dan Validasi
plt.plot(epoch_range, train_acc, color='red', label='Training Accuracy')
plt.plot(epoch_range, val_acc, color='blue', label='Validation Accuracy')
plt.title('Akurasi Pelatihan dan Validasi')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend(loc='upper left')
plt.show()

# Plot Loss Pelatihan dan Validasi
plt.plot(epoch_range, train_loss, color='red', label='Training Loss')
plt.plot(epoch_range, val_loss, color='blue', label='Validation Loss')
plt.title('Loss Pelatihan dan Validasi')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# acc = history_1.history['accuracy']
# val_acc = history_1.history['val_accuracy']
# loss = history_1.history['loss']
# val_loss = history_1.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'r')
# plt.plot(epochs, val_acc, 'b')
# plt.title('Training and Validation Accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

# plt.plot(epochs, loss, 'r')
# plt.plot(epochs, val_loss, 'b')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.title('Training and Validaion Loss')
# plt.show()

#**Akurasi ML**

In [ ]:
# Mengatur ulang test generator
test_generator.reset()

# Melakukan prediksi pada test set
predictions = model_organ.predict(test_generator, verbose=0)
predictions = predictions.copy()

# Mengonversi probabilitas menjadi label biner berdasarkan threshold 0.5
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1

# Menampilkan laporan klasifikasi
from sklearn.metrics import classification_report
print("\nLaporan Klasifikasi:")
print(classification_report(y_true=test_generator.classes, y_pred=predictions, target_names=['Normal', 'Pneumonia'], digits=4))


In [ ]:
# test_generator.reset()

# preds_1 = model_1.predict(test_generator,verbose=0)
# preds_1 = preds_1.copy()
# preds_1[preds_1 <= 0.5] = 0
# preds_1[preds_1 > 0.5] = 1

# # Print Confusion Matrix
# # cm = pd.DataFrame(data=confusion_matrix(test_generator.classes, preds_1, labels=[0, 1]),index=["Actual Normal", "Actual Pneumonia"],
# # columns=["Predicted Normal", "Predicted Pneumonia"])
# # sns.heatmap(cm,annot=True,fmt="d")

# # Print Classification Report
# print("\n")
# print(classification_report(y_true=test_generator.classes,y_pred=preds_1,target_names =['Normal','Pneumonia'], digits=4))

#**Save Model**

In [ ]:
# Mendapatkan class names dari train generator
class_indices = train_generator.class_indices

# Mengubah dictionary class_indices menjadi list class names
class_names = list(class_indices.keys())
print(class_names)

In [ ]:
# Simpan model ke format SavedModel
tf.saved_model.save(model_organ, "saved_model")

#**Mengonversi Model ke TensorFlow Lite**

In [ ]:
model_organ.save('model_saya.keras')

In [ ]:
# Muat model yang telah disimpan dalam format .keras
model = tf.keras.models.load_model('model_saya.keras')


os.makedirs('tflite', exist_ok=True)

# Konversi model ke format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFLite ke file
with open('tflite/model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
!pip install tensorflowjs

In [ ]:
# Konversi SavedModel ke format TFJS
import os
os.makedirs('tfjs_model', exist_ok=True)

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model saved_model tfjs_model/